- 2. Análisis de Ventas en el Boston Housing Dataset
    
    Utilizando el dataset de **Boston Housing**, realiza los siguientes análisis sobre la columna **MEDV** (valor medio de las viviendas en $1000s):
    
    1. **Cálculo de valores extremos**:
        - Obtén el valor **mínimo** y **máximo** de la columna **MEDV**.
        - Calcula el **percentil 25 y 75** de **MEDV**.
        - Calcula el **rango intercuartílico (IQR)** de **MEDV**.
    2. **Detección de valores atípicos**:
        - Encuentra los **valores atípicos** usando la regla de 1.5 * IQR.
        - Imprime cuántos valores atípicos hay y cuáles son.
    3. **Normalización de precios**:
        - Aplica **RobustScaler** para normalizar la columna **MEDV**.
    4. **Clasificación de precios**:
        - Crea una nueva columna que clasifique las viviendas en **"Bajo"**, **"Medio"** o **"Alto"** según los siguientes criterios:
            - "Bajo" si el valor está por debajo del percentil 25.
            - "Medio" si está entre el percentil 25 y 75.
            - "Alto" si está por encima del percentil 75.
    5. **Clasificación por cuartiles**:
        - Usa `qcut` para dividir los precios en **4 categorías** de igual tamaño.

In [81]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.preprocessing import RobustScaler

# Leemos el csv y lo cargamos en un Dataframe (Solamente cargamos la columna medv con la cual vamos a trabajar)
df = pd.read_csv("BostonHousing.csv", usecols=["medv"])
# Creamos un dataFrame en el cual almacenar los valores extremos de la columna medv
df_extremos = pd.DataFrame(columns=["medv"])

# Calculamos los valores extremos y los añadimos como filas al dataframe
df_extremos.loc["min"] = df["medv"].min()
df_extremos.loc["max"] = df["medv"].max()
df_extremos.loc["percentil_25"] = df["medv"].quantile(0.25)
df_extremos.loc["percentil_75"] = df["medv"].quantile(0.75)
df_extremos.loc["iqr"] = df["medv"].quantile(0.75) - df["medv"].quantile(0.25)
# Imprimimos por pantalla el dataframe
print(tabulate(df_extremos, headers="keys", tablefmt="psql"))

# Detección de outlayers mediante el rango intercuartílico
Q1 = df_extremos["medv"]["percentil_25"]
Q3 = df_extremos["medv"]["percentil_75"]
IQR = df_extremos["medv"]["iqr"]
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df["es_outlier"] = (df["medv"] < lower_bound) | (df["medv"] > upper_bound)
outlayers = df.query("es_outlier")["medv"].to_list()

print("-" * 30)
print("Cantidad de outlayers: ", len(outlayers))
print("Outlayers: ", outlayers)
print("-" * 30)

# Aplicar RobutScaler para normalizar la columna medv
scaler = RobustScaler()
array = np.array(
    df["medv"]
).reshape(
    -1, 1
)  # Pasamos la columna a un array bidimensional para que la función de normalización la acepte como parámetro
df["normalize_medv"] = scaler.fit_transform(array)


# Creación de una nueva columna clasificación de viviendas mediante la aplicación de una función a la columna 'mdev'
def clasificar_viviendas(data):
    if data < Q1:
        return "Bajo"
    elif Q1 < data < Q3:
        return "Medio"
    else:
        return "Alto"


df["medv_clasification"] = df["medv"].apply(clasificar_viviendas)

# Clasificación por cuartiles
df["mdev_cuartiles"] = pd.qcut(
    df["medv"], q=4, labels=["bajo", "medio-bajo", "medio-alto", "alto"]
)

# Impresión del dataframe por pantalla
print(tabulate(df, headers="keys", tablefmt="psql"))


+--------------+--------+
|              |   medv |
|--------------+--------|
| min          |  5     |
| max          | 50     |
| percentil_25 | 17.025 |
| percentil_75 | 25     |
| iqr          |  7.975 |
+--------------+--------+
------------------------------
Cantidad de outlayers:  40
Outlayers:  [38.7, 43.8, 41.3, 50.0, 50.0, 50.0, 50.0, 37.2, 39.8, 37.9, 50.0, 37.0, 50.0, 42.3, 48.5, 50.0, 44.8, 50.0, 37.6, 46.7, 41.7, 48.3, 42.8, 44.0, 50.0, 43.1, 48.8, 50.0, 43.5, 45.4, 46.0, 50.0, 37.3, 50.0, 50.0, 50.0, 50.0, 50.0, 5.0, 5.0]
------------------------------
+-----+--------+--------------+------------------+----------------------+------------------+
|     |   medv | es_outlier   |   normalize_medv | medv_clasification   | mdev_cuartiles   |
|-----+--------+--------------+------------------+----------------------+------------------|
|   0 |   24   | False        |        0.351097  | Medio                | medio-alto       |
|   1 |   21.6 | False        |        0.0501567 | Med